In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25243355
paper_name = 'paget_delneri_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [11]:
original_data = pd.read_excel('raw_data/mec12930-sup-0011-tables8.xlsx', sheet_name='Sheet1')

In [12]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 6004 x 4


In [13]:
original_data.head()

,ORF,YPD,Carbon limited,Nitrogen limited
0,YMR194W,2.140680,3.619595,0.335091
1,YBR286W,2.147900,2.225514,-1.046375
2,YMR198W,1.570476,NaN,0.794221
3,YDR094W,-0.283968,0.424738,0.253968
4,YOL131W,-0.551980,-0.843094,7.028037


In [14]:
original_data['orf'] = original_data['ORF'].astype(str)

In [15]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [16]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [17]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                    ORF  YPD  Carbon limited  Nitrogen limited        orf
index_input                                                              
5752         YBR160W_AS  NaN        1.287126               inf  YBR160WAS


In [18]:
original_data = original_data.loc[t,:]

In [19]:
original_data.set_index('orf', inplace=True)

In [20]:
original_data = original_data[['YPD','Carbon limited','Nitrogen limited']].copy()

In [21]:
original_data = original_data.groupby(original_data.index).mean()

In [22]:
original_data.shape

(5898, 3)

In [31]:
original_data[np.isinf(original_data)] = np.nan

# Prepare the final dataset

In [32]:
data = original_data.copy()

In [33]:
dataset_ids = [22234, 22235, 22236]
datasets = datasets.reindex(index=dataset_ids)

In [34]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [35]:
data.head()

dataset_id,22234,22235,22236
data_type,value,value,value
orf,,,
YAL001C,-0.527029,-0.824125,0.072327
YAL002W,0.492276,0.586086,1.113728
YAL003W,-0.525214,-0.068258,0.309637
YAL004W,-0.725044,-0.618899,0.733940
YAL005C,0.687833,1.222700,0.562115


## Subset to the genes currently in SGD

In [36]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [37]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22234,22235,22236
,data_type,value,value,value
gene_id,orf,,,
1,YAL001C,-0.527029,-0.824125,0.072327
2,YAL002W,0.492276,0.586086,1.113728
3,YAL003W,-0.525214,-0.068258,0.309637
1863,YAL004W,-0.725044,-0.618899,0.733940
4,YAL005C,0.687833,1.222700,0.562115


# Normalize

In [38]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [39]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [40]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,22234,22235,22236,22234,22235,22236
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
1,YAL001C,-0.527029,-0.824125,0.072327,0.309464,-0.300036,0.069665
2,YAL002W,0.492276,0.586086,1.113728,1.467789,0.946411,0.780277
3,YAL003W,-0.525214,-0.068258,0.309637,0.311526,0.368054,0.231597
1863,YAL004W,-0.725044,-0.618899,0.733940,0.084442,-0.118643,0.521125
4,YAL005C,0.687833,1.222700,0.562115,1.690017,1.509098,0.403878


# Print out

In [55]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [60]:
# from IO.save_data_to_db3 import *

In [61]:
# save_data_to_db(data_all, paper_pmid)